# Agent-based model for News diffusion and misinformation

What I want to do is to create agents that share news, then from there find some interesting insights that could help fake news research    
Test 1 Does having fake news affect real news sharing?    
Test 2 Does a news source matter in the design?    
Test 3 Does a news event matter in the design?    
Test 4 Do people just share what they are told (Illusory of truth)?    
Test 5 How people could be silenced?    
Test 6 Does fake news shift the public opinion?

#### The first thing we need to do is to import news article and user classes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from SimHelper import generate_weighted_sentiment
from random import sample, randint
from tqdm import tqdm_notebook as tqdm
from classes import NewsArticle,User, NewsAgency
import pylab
import networkx as nx
from random import random, uniform, choice
from SimHelper import generate_weighted_percentage, generate_bias
from scipy.stats import beta
from random import sample
import collections
import scipy.stats
import itertools
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error

In [2]:
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 16})

In [3]:
def convert_source_to_list(source):
    y = [0]*number_of_news_users
    y[source] = 1
    return y

def create_news_event (user,sentiment,fake,tick):
    """
    Make a news event
    a. news_article_id
    b. source_id : the id of the source
    c. source_preference : the preference of the source
    d. sentiment : a number from [1,5] or [-5,-1]
    e. num_shares : the number of people sharing the article
    f. fake : 1 if the news is fake, otherwise real
    """
    # assign an id in the format of user id + 1000
    if not user.articles:
        news_id = int(str(user.unique_id)+str(1000))
    else:
        news_id = user.articles[-1].news_article_id +1
#     print(news_id)
    # get source_id and source_preference
    source_id = user.unique_id
    source_preference = user.preference
    #generate a preference randomly distributed around the source preference
    if fake:
        article_preference = round(np.random.normal(loc=source_preference, scale = 0.14),2)
        if np.sign(article_preference) != np.sign(user.preference):
            article_preference *= -1
    else:
        article_preference = round(np.random.normal(loc=source_preference, scale = 0.07),2)
    if article_preference > 1:
        article_preference = 1
    elif article_preference < -1:
        article_preference = -1
    sentiment = sentiment
    num_shares = 1
    fake = fake
    tick = tick
    
    article = NewsArticle(news_id,source_id,source_preference,article_preference,sentiment,num_shares,fake,tick)
    
    user.articles.append(article)

def get_similarity(x,y):
    return round(1 - abs(G.nodes[x]['preference'] - G.nodes[y]['preference'])/2,2)

def get_neighbor_preferences(user):
    """
    Get the average preference of a user's neighbors
    """
    return round(np.mean([G.nodes[n]['preference'] for n in list(G[user.unique_id])]),2)
#     return round(np.mean([o.preference for o in list(G[users[user.unique_id]])]),2)

    # Simpler version
#     temp = []
#     for n in list(G[users[user.unique_id]]):
#         print(abs(user.preference - n.preference))
#         temp.append(n.preference)
#     return round(np.mean(temp),2)

def get_recommendations(user,tick):
    """
    Pick what the user will see on their feed according to a platform strategy
    
    Strategy 1: See the latest news from your neighbors
    Strategy 2: See the most popular news
    Strategy 3: See the most important news in your circle
    Strategy 4: See a random selected news as a benchmark
    Strategy 5: Weighted approach
    Strategy 6: Most shocking news
    """
    recommendations = [article for neighbor in list(G[user.unique_id]) for article in G.nodes[neighbor]['articles'] if article.tick > tick -5]
    recommendations = [article for article in recommendations if np.sign(user.preference) == np.sign(article.article_preference)]
    # try to not recommend articles that look like fake news
#     print(article_sentiments_mean)
#     recommendations = [article for article in recommendations if (abs(article.sentiment) < sentiment_threshold and article.num_shares > int(num_shares_n))or random()>0.5]
#     recommendations.sort(key=lambda x: (x.tick,abs(x.sentiment)), reverse=True)

    #using a fake news classifier
    if (tick > 0) and (tick/pause > pause/tick) :
        for rec in recommendations:
            if clf.predict([[rec.sentiment,rec.num_shares]+convert_source_to_list(rec.source_id)])[0] == 1:
                    recommendations.remove(rec)
    
    try:
        return recommendations[:3]
    except:
        return recommendations

def spread_news(user,tick):
    """
    A user will share to their neighbors based on preference, sentiment, and number of shares
    
    In terms of preference, there are a few measures to find:
    1- individual preferences: user preference and neighbor preference
    2- community preference : mean of preference of neighbors of the neighbor
    3- 
    """
#     if not user.articles:
#         return
    
    community_opinion = get_neighbor_preferences(user)
    recommendations = get_recommendations(user,tick)
    
    for rec in recommendations:
        if rec.fake and (random() < accept_fact_checking):
            #user caught the fact checking signal
            continue
        # person likes to share
        if generate_weighted_percentage(user.news_spread_chance):
            #person is not silenced
            if np.sign(community_opinion) == np.sign(user.preference) == np.sign(rec.article_preference):
                if rec not in user.articles and user.user_type =='regular':
                    user.articles.append(rec)
                    spreaders.loc[len(spreaders.index)] = [tick, user.unique_id,rec.source_id,rec.sentiment, rec.num_shares, rec.fake]
                    rec.num_shares += 1
                    preference_shift = 0.03
#                     print(user.unique_id,'before update:',users[user.unique_id].preference)
                    users[user.unique_id].preference = round(users[user.unique_id].preference,2)
                    if (np.sign(user.preference) > 0):
                        if users[user.unique_id].preference > rec.article_preference:
                            users[user.unique_id].preference -= preference_shift
#                             print(user.unique_id,'after update:',users[user.unique_id].preference)
                        elif users[user.unique_id].preference < rec.article_preference:
                            users[user.unique_id].preference += preference_shift
#                             print(user.unique_id,'after update:',users[user.unique_id].preference)
                    elif (np.sign(user.preference) < 0):
                        if users[user.unique_id].preference > rec.article_preference:
                            users[user.unique_id].preference -= preference_shift
#                             print(user.unique_id,'after update:',users[user.unique_id].preference)
                        elif users[user.unique_id].preference < rec.article_preference:
                            users[user.unique_id].preference += preference_shift
#                             print(user.unique_id,'after update:',users[user.unique_id].preference)
                    if users[user.unique_id].preference > 1:
                        users[user.unique_id].preference = 1
                    elif users[user.unique_id].preference < -1:
                        users[user.unique_id].preference = -1

def get_regular_user_preferences():
    """
    return the mean and std dev of the user preference
    """
#     preferences = []
#     for node in regular_users:
#         preferences.append(node.preference)

    preferences = []
    for node in users:
        preferences.append(node.preference)

    print('mean = ',round(np.mean(preferences),2),'standard dev = ',round(np.std(preferences),2))
    plt.hist(preferences)

def clear_all_news():
    [user.articles.clear() for user in users]
    
def train_fake_news_classifier(verbose=0):
#     df = pd.get_dummies(spreaders,columns=['source'])
    df = spreaders.copy()
    sources_df = []
    for i in range(number_of_news_users):
        sources_df.append('source_'+str(i))
    sources_df

    df = pd.concat([df,pd.DataFrame(columns=sources_df)],sort=False)
    df.fillna(0,inplace=True)
    for index,row in df.iterrows():
        column_name = 'source_'+str(int(row['source']))
        df.loc[index,column_name]  = 1
    df.drop(columns=['source'],inplace=True)
    df = df[[c for c in df if c not in ['fake']]+['fake']]
    clf = svm.SVC(gamma='scale')
    clf.fit(df.drop(columns=['tick','user','fake']), df['fake'])
#     print(df.drop(columns=['tick','user','fake']))
    if verbose >= 1:
        y_pred = clf.predict(df.drop(columns=['tick','user','fake']))
        print('MSE',mean_squared_error(df['fake'].to_numpy(),y_pred))
        if verbose == 2:
            print(confusion_matrix(df['fake'].to_numpy(),y_pred))
            print(classification_report(df['fake'].to_numpy(),y_pred))
    return clf

In [4]:
# df = spreaders.copy()
# sources_df = []
# for i in range(number_of_news_users):
#     sources_df.append('source_'+str(i))
# sources_df

# df = pd.concat([df,pd.DataFrame(columns=sources_df)],sort=False)
# df.fillna(0,inplace=True)
# for index,row in df.iterrows():
#     column_name = 'source_'+str(int(row['source']))
#     df.loc[index,column_name]  = 1
# df.drop(columns=['source'],inplace=True)

1. User attributes:  
  a. unique_id : a non-intelligent number given to the user  
  b. news_spread_chance : a percentage of the user spreading news (can be compared to confidence and censorship)   
  c. preference : the position the user feels about the current subject being discussed  
  d. user_type: the user could be a regular user or a news agency  
  e. articles : a list of articles shared by the user  



2. News agency attributes:  
  a. unique_id : a non-intelligent number given to the user  
  b. news_spread_chance : a percentage of the user spreading news (can be compared to confidence and censorship)   
  c. preference : the position the user feels about the current subject being discussed  
  d. user_type: the user could be a regular user or a news agency  
  e. articles : a list of articles shared by the user  
  f. reliable : indicates how reliable this agency is
        
        
3. News article attributes:  
  a. news_article_id : a non-intelligenct number given to the article  
  b. source_id : the id of the source  
  c. source_preference : the preference of the source  
  d. sentiment : a number from [1,5] or [-5,-1]  
  e. num_shares :  the number of people sharing the article  
  f. fake : 1 if the news is fake, otherwise real

#### Lets try to create a simple news article

#### Now we need to try to create a few users

news_agencies_ratio is a network attribute to add that takes the amount of news agencies in the network  
number_of_users is a network attribute that creates a graph of a certain number of nodes equal to the number of users

### Lets try to create a network of the users

#### Add edges by Power law distribution

#### Lets spread some news and see how users change their preferences

ticks is another network variable that is used to measure time, it does not mean a specfic time stamp we know at the moment  
fake_news_prob is another network variable that measures how much the percentage of fake news will be generated throughout the network  
For every time step t:  
- Make news events from the news agencies  
- Ask every agent to spread their articles across the network  
- Change the 'regular' user preference and compute overall network preference

In [5]:
number_of_users = 100
news_agencies_ratio = 0.1
news_reliability = 0.1
ticks = 100
pause = int(0.2 * ticks)
fake_news_prob = 0.5
accept_fact_checking = 0.999
sentiment_threshold = 4.1
num_shares_n= 3

mu = {new_list: [] for new_list in range(0,ticks,pause)} 
si = {new_list: [] for new_list in range(0,ticks,pause)} 
fk = []
rl= []
all_news = []

for z in range(50):
    
    users = []
    regular_users = []
    news_users = []
    


    number_of_regualr_users = int(number_of_users*(1-news_agencies_ratio))
    number_of_news_users = number_of_users - number_of_regualr_users

    
    #create the distribution for news agencies to be two concatenatened beta distribution with alpha=beta=3, one of them shifted by -1
    data_beta_neg = beta.rvs(a=3,b=3,size=1000)
    data_beta_neg = [x-1 for x in data_beta_neg]
    
    data_beta_pos = list(beta.rvs(a=3,b=3,size=1000))
    
    beta_dist = data_beta_neg + data_beta_pos


    for i in range(number_of_news_users):
        unique_id = i
        user_type = 'news_agency' 
        news_spread_chance = 1
        articles = []
        reliable = (generate_weighted_percentage(news_reliability))
        if (reliable == 0) and (random() >0.5):
            preference = round(choice([x for x in beta_dist if abs(x)>0.4]),2)
        else:
            preference = round(choice([x for x in beta_dist if abs(x)<0.5]),2)
        news_users.append(NewsAgency(unique_id,news_spread_chance,preference,user_type,articles,reliable))
        users.append(NewsAgency(unique_id,news_spread_chance,preference,user_type,articles,reliable))

    for i in range(len(news_users),number_of_regualr_users+len(news_users)):
        unique_id = i
        user_type = 'regular'
        news_spread_chance = round(random(),2)
        preference = generate_bias(mu = 0, sigma = 3)    
        articles = []
        regular_users.append(User(unique_id,news_spread_chance,preference,user_type,articles))
        users.append(User(unique_id,news_spread_chance,preference,user_type,articles))

    G = nx.Graph()
    color_map = []
    for u in users:    
        if isinstance(u,NewsAgency):
            G.add_node(u.unique_id,s="^", news_spread_chance = u.news_spread_chance, 
                       preference = u.preference, reliable= u.reliable, articles = u.articles, user_type = u.user_type )
        else:
            G.add_node(u.unique_id,s="o", news_spread_chance = u.news_spread_chance, 
                       preference = u.preference, articles = u.articles, user_type = u.user_type )
        if u.preference < -0.2:
            color_map.append('red')
        elif u.preference > 0.2:
            color_map.append('blue')
        else:
            color_map.append('grey') 
            
            
    edge_list = []
    for i in range(number_of_users):
        number_of_edges = int((np.random.pareto(a=5)+.1)*30)
        if number_of_edges > number_of_users:
            number_of_edges /= 2
        edge_list.append(int(number_of_edges))
        
    
    for i in range(number_of_users):
        # I am writing the sample function in while True loop to make sure it happens
        while True:
            try:
                neighbors = sample(users,edge_list[i])
            except:
                continue
            break
    #     print(neighbors)
        for neighbor in neighbors:
            u = users[i].unique_id
            v = neighbor.unique_id
            s = get_similarity(u,v)
            G.add_edge(u,v,weight=s)  
            
    clear_all_news()

#     fig,axs = plt.subplots(int(ticks/pause),figsize=(25,25))
#     fig.suptitle('Regular users preferences over time')
#     plots = 0
    spreaders = pd.DataFrame(columns=['tick','user','source','sentiment','num_shares','fake'])
    for tick in range(ticks):
        if tick % pause == 0:
#             print('tick number: ',tick)
    #         get_regular_user_preferences()
            preferences = [user.preference for user in users if user.user_type == 'regular']
            mu[tick].append(np.mean(preferences))
            si[tick].append(np.std(preferences))
            # train the classifier
            if tick>0:
                clf = train_fake_news_classifier()
#             print('mean = ',round(np.mean(preferences),2),'standard dev = ',round(np.std(preferences),2))
            
#             axs[plots].hist(preferences)
#             plots += 1

        sampled_news_agencies = sample(news_users,randint(1,len(news_users)-1))
        for n in sampled_news_agencies:
            sentiment = generate_weighted_sentiment()
            fake = 0
            if n.reliable == 0:
                if generate_weighted_percentage(fake_news_prob):
                    #generate fake news
                    sentiment = generate_weighted_sentiment(a=4)
                    fake = 1
            create_news_event(n,sentiment,fake,tick)
            spread_news(n,tick)
        sampled_regular_users = sample(regular_users,randint(1,len(regular_users)-1))
        for u in sampled_regular_users:
            spread_news(u,tick)
        
    ## store news in a df to identify spammers    
#     print('news spreaders',len(spreaders))
#     print(spreaders.head())
            
    w = []
    for u in users:
        if u.articles:
            for a in u.articles:
                w.append(a)

    w.sort(key=lambda x: x.num_shares, reverse=True)

    getx, gety = lambda a: a.num_shares, lambda a: a.news_article_id # or use operator.itemgetter
    groups = itertools.groupby(sorted(w, key=getx), key=getx)
    m = [max(b, key=gety) for a,b in groups]
    copy = [l for l in w if l in m]

    copy.sort(key=lambda x: x.num_shares, reverse=True)
    
    
    all_news = all_news + copy
    
    fk.append(sum([x.num_shares for x in copy if x.fake]))

    rl.append(sum([x.num_shares for x in copy if not x.fake]))
    print('finished round:',z)

ValueError: The number of classes has to be greater than one; got 1 class

In [ ]:
shares_fake=[x.num_shares for x in all_news if x.fake == 1]
shares_real=[x.num_shares for x in all_news if x.fake == 0]

print(sum(shares_fake)/(sum(shares_fake)+sum(shares_real)))

In [6]:
# fig, ax = plt.subplots()
plt.figure(figsize=(10,10))
plt.hist(shares_real,alpha = 0.5, color = 'green',label='Real')
plt.hist(shares_fake,alpha = 0.7, color = 'red',label='Fake')
plt.legend(loc='upper right')
plt.title('News sharing results with a fake news classifier')
plt.show()

In [ ]:
fake_news_percentage = {}
for senti_t in range(49,40,-2):
    for num_shares_n in range(3,16,3):
        number_of_users = 100
        news_agencies_ratio = 0.1
        news_reliability = 0.1
        ticks = 100
        pause = int(0.2 * ticks)
        fake_news_prob = 0.5
        accept_fact_checking = 1
        sentiment_threshold = senti_t/10
        num_shares_threshold = num_shares_n

        mu = {new_list: [] for new_list in range(0,ticks,pause)} 
        si = {new_list: [] for new_list in range(0,ticks,pause)} 
        fk = []
        rl= []
        all_news = []

        for z in range(50):

            users = []
            regular_users = []
            news_users = []

            number_of_regualr_users = int(number_of_users*(1-news_agencies_ratio))
            number_of_news_users = number_of_users - number_of_regualr_users


            #create the distribution for news agencies to be two concatenatened beta distribution with alpha=beta=3, one of them shifted by -1
            data_beta_neg = beta.rvs(a=3,b=3,size=1000)
            data_beta_neg = [x-1 for x in data_beta_neg]

            data_beta_pos = list(beta.rvs(a=3,b=3,size=1000))

            beta_dist = data_beta_neg + data_beta_pos


            for i in range(number_of_news_users):
                unique_id = i
                user_type = 'news_agency' 
                news_spread_chance = 1
                articles = []
                reliable = (generate_weighted_percentage(news_reliability))
                if (reliable == 0) and (random() >0.5):
                    preference = round(choice([x for x in beta_dist if abs(x)>0.4]),2)
                else:
                    preference = round(choice([x for x in beta_dist if abs(x)<0.5]),2)
                news_users.append(NewsAgency(unique_id,news_spread_chance,preference,user_type,articles,reliable))
                users.append(NewsAgency(unique_id,news_spread_chance,preference,user_type,articles,reliable))

            for i in range(len(news_users),number_of_regualr_users+len(news_users)):
                unique_id = i
                user_type = 'regular'
                news_spread_chance = round(random(),2)
                preference = generate_bias(mu = 0, sigma = 3)    
                articles = []
                regular_users.append(User(unique_id,news_spread_chance,preference,user_type,articles))
                users.append(User(unique_id,news_spread_chance,preference,user_type,articles))

            G = nx.Graph()
            color_map = []
            for u in users:    
                if isinstance(u,NewsAgency):
                    G.add_node(u.unique_id,s="^", news_spread_chance = u.news_spread_chance, 
                               preference = u.preference, reliable= u.reliable, articles = u.articles, user_type = u.user_type )
                else:
                    G.add_node(u.unique_id,s="o", news_spread_chance = u.news_spread_chance, 
                               preference = u.preference, articles = u.articles, user_type = u.user_type )
                if u.preference < -0.2:
                    color_map.append('red')
                elif u.preference > 0.2:
                    color_map.append('blue')
                else:
                    color_map.append('grey') 


            edge_list = []
            for i in range(number_of_users):
                number_of_edges = int((np.random.pareto(a=5)+.1)*30)
                if number_of_edges > number_of_users:
                    number_of_edges /= 2
                edge_list.append(int(number_of_edges))


            for i in range(number_of_users):
                # I am writing the sample function in while True loop to make sure it happens
                while True:
                    try:
                        neighbors = sample(users,edge_list[i])
                    except:
                        continue
                    break
            #     print(neighbors)
                for neighbor in neighbors:
                    u = users[i].unique_id
                    v = neighbor.unique_id
                    s = get_similarity(u,v)
                    G.add_edge(u,v,weight=s)  

            clear_all_news()

        #     fig,axs = plt.subplots(int(ticks/pause),figsize=(25,25))
        #     fig.suptitle('Regular users preferences over time')
        #     plots = 0
            spreaders = pd.DataFrame(columns=['tick','user','source','sentiment','num_shares','fake'])
            for tick in range(ticks):
                if tick % pause == 0:
        #             print('tick number: ',tick)
            #         get_regular_user_preferences()
                    preferences = [user.preference for user in users if user.user_type == 'regular']
                    mu[tick].append(np.mean(preferences))
                    si[tick].append(np.std(preferences))
                    # train the classifier
    #                 if tick>0:
    #                     clf = train_fake_news_classifier()
        #             print('mean = ',round(np.mean(preferences),2),'standard dev = ',round(np.std(preferences),2))

        #             axs[plots].hist(preferences)
        #             plots += 1

                sampled_news_agencies = sample(news_users,randint(1,len(news_users)-1))
                for n in sampled_news_agencies:
                    sentiment = generate_weighted_sentiment()
                    fake = 0
                    if n.reliable == 0:
                        if generate_weighted_percentage(fake_news_prob):
                            #generate fake news
                            sentiment = generate_weighted_sentiment(a=4)
                            fake = 1
                    create_news_event(n,sentiment,fake,tick)
                    spread_news(n,tick)
                sampled_regular_users = sample(regular_users,randint(1,len(regular_users)-1))
                for u in sampled_regular_users:
                    spread_news(u,tick)

            ## store news in a df to identify spammers    
        #     print('news spreaders',len(spreaders))
        #     print(spreaders.head())

            w = []
            for u in users:
                if u.articles:
                    for a in u.articles:
                        w.append(a)

            w.sort(key=lambda x: x.num_shares, reverse=True)

            getx, gety = lambda a: a.num_shares, lambda a: a.news_article_id # or use operator.itemgetter
            groups = itertools.groupby(sorted(w, key=getx), key=getx)
            m = [max(b, key=gety) for a,b in groups]
            copy = [l for l in w if l in m]

            copy.sort(key=lambda x: x.num_shares, reverse=True)


            all_news = all_news + copy

            fk.append(sum([x.num_shares for x in copy if x.fake]))

            rl.append(sum([x.num_shares for x in copy if not x.fake]))
            print('finished round:',z)
        shares_fake=[x.num_shares for x in all_news if x.fake == 1]
        shares_real=[x.num_shares for x in all_news if x.fake == 0]
        fn_percentage = sum(shares_fake)/(sum(shares_fake)+sum(shares_real))
        fake_news_percentage[(sentiment_threshold,num_shares_n)] = fn_percentage
        print(sentiment_threshold,num_shares_n,":",fn_percentage)

finished round: 0
finished round: 1
finished round: 2
finished round: 3
finished round: 4
finished round: 5
finished round: 6
finished round: 7
finished round: 8
finished round: 9
finished round: 10
finished round: 11
finished round: 12
finished round: 13
finished round: 14
finished round: 15
finished round: 16
finished round: 17
finished round: 18
finished round: 19
finished round: 20
finished round: 21
finished round: 22
finished round: 23
finished round: 24
finished round: 25
finished round: 26
finished round: 27
finished round: 28
finished round: 29
finished round: 30
finished round: 31
finished round: 32
finished round: 33
finished round: 34
finished round: 35
finished round: 36
finished round: 37
finished round: 38
finished round: 39
finished round: 40
finished round: 41
finished round: 42
finished round: 43
finished round: 44
finished round: 45
finished round: 46
finished round: 47
finished round: 48
finished round: 49
4.9 3 : 0.5346425237493823
finished round: 0
finished round:

In [ ]:
fake_news_percentage

In [226]:
fake_news_percentage

{4.9: 0.5483019594135505, 4.5: 0.5558534796931259, 4.1: 0.5113095890410959}

In [117]:
print(np.mean(mu[0]),np.mean(mu[100]),np.mean(mu[200]),np.mean(mu[300]),np.mean(mu[400]))

-0.007855555555555555 -0.007722222222222222 -0.0021888888888888874 -0.0050555555555555545 -0.010322222222222222


In [121]:
print(np.mean(si[0]),np.mean(si[100]),np.mean(si[200]),np.mean(si[300]),np.mean(si[400]))

0.298053701786515 0.5666194816981419 0.5883223133197757 0.5887259524968597 0.5938585493413071


In [122]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [123]:
mean_confidence_interval(si[0])

(0.298053701786515, 0.2920978124635251, 0.30400959110950493)

In [124]:
np.mean(fk)

4595.333333333333

In [125]:
np.mean(rl)

2780.9583333333335

In [127]:
np.mean(fk)/(np.mean(fk)+np.mean(rl))

0.6229869344916992